# Numerical weather prediction data
The following notebook elaborates on how to use the TWIGA API to retrieve and work with weather forecast data. Examples are based on the Unified Model for Southern Africa (below the equator).

In [1]:
#############################
# Load some libraries       #
#############################
import shutil
import requests
import json
import datetime
import pandas as pd
import matplotlib
import numpy as np
import seaborn as sns

# plot figures directly into the notebook
%matplotlib inline

In [2]:
#############################
# TWIGA API settings        #
#############################

# The HydroNET/TWIGA API endpoint
api = 'https://hnapi.hydronet.com/api/'

# The bearer token for the TWIGA user
# this token is used to identify as a valid TWIGA user to the API
api_token = 'eyJhbGciOiJSUzI1NiIsImtpZCI6ImIyY2I4NjU2NjNlY2RiYzEyMGZkOGViYzFkM2ExOGIwIiwidHlwIjoiSldUIn0.eyJuYmYiOjE1NzE4MzEzOTksImV4cCI6MTcyMzA2MjYyOSwiaXNzIjoiaHR0cHM6Ly9vaWRjLmh5ZHJvbmV0LmNvbSIsImF1ZCI6Imh0dHBzOi8vb2lkYy5oeWRyb25ldC5jb20vcmVzb3VyY2VzIiwiY2xpZW50X2lkIjoiaG40cy1wcm9kdWN0aW9uIiwiY2xpZW50X3Byb2ZpbGUiOiJwcm9maWxlIiwiY2xpZW50X2VtYWlsIjoiZW1haWwiLCJjbGllbnRfbmFtZSI6Im5hbWUiLCJjbGllbnRfc3RyaW5nIjoiaWRfdG9rZW4gdG9rZW4iLCJjbGllbnRfb2ZmbGluZV9hY2Nlc3MiOiJvZmZsaW5lX2FjY2VzcyIsInN1YiI6IjQzYmEwNjhjLTI3NzYtNDg1My1hYjc5LTIyMzIxMTE3NzZhMCIsImF1dGhfdGltZSI6MTU3MTgzMTM5OSwiaWRwIjoibG9jYWwiLCJqdGkiOiIyNTg1M2UwNjRjMGFhYjg2N2RmZTgyNjdjNzBmZTcyMCIsInNjb3BlIjpbIm9wZW5pZCIsInByb2ZpbGUiLCJvZmZsaW5lX2FjY2VzcyJdLCJhbXIiOlsicHdkIl19.PhkJ3SVi2ZjBEan9OlV9qpFAV3fqjsPQqQv-gk_ZtWlIuwTvoPMt5whwAa07opUNV5tO_-Jk4B8R-W9x2znMmzA_d9Bjyiwrwto3GgaUBBryXR-GIs21Dy1Hj62qUvUDpI07EQhGeq7SpxYeO0WdK_t-5U-3w3y9WNihvmcyfMsJukw9AOsSObEPUY6YUiTv71vbqKPQc55pbuPHpaDRHZITH0Nps3E_jrnwn9Aepz7B7MDVXyM_vU1Vb-MzAVZq03XvCf_YdWWxQUqPhbBVHcJjfzSQGrFl-8pGMLkHvcx1PgEnrxN9y7JS6tP_D7rZMBwA4vaiHXX5Z1XdBsvwYg'

# Using the token, generate a valid header for requests to the API
api_header = {'content-type': 'application/json', 'Authorization': 'bearer ' + api_token}


# Accessing the available data sources
The TWIGA api holds a lot of data. There are different data sources available, each containing relevant data for TWIGA. Using the API one can request data of any of the TWIGA data sources. If you know which datasource you are intersted in, you can direct request data from iw. However, often one of the first steps is to look into what data sources are available.

Getting an overview of the different data sources can be done by sending a request to the 'datasources' endpoint of the TWIGA API.

In [3]:
# an empty request which can be send to the TWIGA API
datasource_metadata = {}

# Send the request to the datasources endpoint of the API
datasource_response = requests.post(api + 'entity/datasources/get', headers=api_header, data=json.dumps(datasource_metadata))

# The response of the API is in JSON. Parse this with Python
datasource_metadata = datasource_response.json()

# print the result, as indented json
print(json.dumps(datasource_metadata, indent=2))


{
  "DataSources": {
    "Rain4Africa.SouthAfrica.Ecmwf.Deterministic.3h": {
      "Name": "ECMWF DET 5d 0.1\u00b0 3h SA",
      "Code": "Rain4Africa.SouthAfrica.Ecmwf.Deterministic.3h",
      "Settings": {
        "StoreInterval": "Days",
        "Time": {
          "Interval": {
            "Type": "Hours",
            "Value": 3.0
          }
        },
        "Grid": {
          "GridDefinitionId": 14
        },
        "Model": {
          "Interval": {
            "Type": "Hours",
            "Value": 12.0
          },
          "TimeStepCount": 41,
          "StartDateOffsetMinutes": -180
        },
        "FileFormatId": 3,
        "SplitByVariable": true,
        "Compression": true,
        "DeflateLevel": 5,
        "ChunkSizeX": 10,
        "ChunkSizeY": 10,
        "ChunkSizeModels": 1,
        "ChunkSizeTime": 1,
        "MaximumDelay": {
          "Type": "Days",
          "Value": 1
        }
      },
      "StartDate": "20160308000000",
      "EndDate": "201911120000

# Questions
Question 1, How many datasources are available in the TWIGA platform?
hint, use the len() function of python

In [4]:
# Answer space
print("There are ",len(datasource_metadata["DataSources"])," Data Sources")


There are  10  Data Sources


In [5]:
# From the response of the TWIGA API we can see that 
# one data source has the code Rain4Africa.SouthernAfrica.UnifiedModel.Deterministic.4km
# this datasource is the NWP data below the equator
# for the remainder of this notebook we will focus on this data source

# store the selected data source code
selected_datasource_code = "Rain4Africa.SouthernAfrica.UnifiedModel.Deterministic.4km"

In [6]:
# json request to ask metadata of a single datasource
request_metadata_um = {
     "DataSourceCodes": [selected_datasource_code]
}

# Send the request to the datasources endpoint of the API
datasource_um_response = requests.post(api + 'entity/datasources/get', headers=api_header, data=json.dumps(request_metadata_um))

# The response of the API is in JSON. Parse this with Python
datasource_um_metadata = datasource_um_response.json()

# print the result, as indented json
print(json.dumps(datasource_um_metadata, indent=2))

{
  "DataSources": {
    "Rain4Africa.SouthernAfrica.UnifiedModel.Deterministic.4km": {
      "Name": "UM DET 3d 4km 1h SADC",
      "Code": "Rain4Africa.SouthernAfrica.UnifiedModel.Deterministic.4km",
      "Settings": {
        "ChunkSizeX": 10,
        "ChunkSizeY": 10,
        "ChunkSizeTime": 1,
        "ChunkSizeModels": 1,
        "Compression": true,
        "DeflateLevel": 5,
        "StoreInterval": "Days",
        "FileFormatId": 3,
        "SplitByVariable": true,
        "SplitByEnsemble": false,
        "SplitByModelTimeStep": false,
        "SplitByLocation": false,
        "SplitByUnit": false,
        "FileDateOffSetInMinutes": 0,
        "Grid": {
          "GridDefinitionId": 27,
          "Pyramid": {
            "MaxPixelCount": 80000,
            "PreferredPixelCount": 80000,
            "Interpolation": "Average"
          }
        },
        "AccumulationInterval": "Total",
        "Disclaimer": null,
        "Time": {
          "Interval": {
            "Type"

# Questions
Question 2, what is the temporal resolution of the Unified Model data?

Question 3, how many forecast timesteps are there in a modelrun?

Question 4, can you determine how often the UM is run per day?


In [16]:
# Answers space
def UnifiedModelSettings(typ):
    timeStep,UM = 0,0

    for dic in datasource_um_metadata["DataSources"]:
        settings = datasource_um_metadata["DataSources"][dic].get("Settings").get(typ)
        
        interval = settings.get("Interval")
        timeType = interval.get("Type")
        time = interval.get("Value")
        if typ == "Model":
            timeStep = settings.get("TimeStepCount")
            UM = int(24/time)
    return timeType, time, timeStep,UM

print(UnifiedModelSettings("Time"))
print(UnifiedModelSettings("Model"))


('Hours', 1.0, 0, 0)
('Hours', 12.0, 73, 2)


In [ ]:
# Numerical weather models are continously updated. When we are going to request data, we want to
# request data of the latest available model run

# The latest available modelrun can be found in the previous response of the API
# it is defined as the 'EndDate' of the data source

# It is of course possible to write this down manually, but we can extract it directly
# from the JSON response

# the JSON response is nested, we can use this structure to get the value from 'EndDate'
datasource_enddate_string = datasource_metadata['DataSources'].get(selected_datasource_code).get('EndDate')

# the enddate is given as a string (text), in YYYYmmddHHMMSS
# if we convert it into an actual datetime object, we can easily work with it in python
datasource_enddate = datetime.datetime.strptime(datasource_enddate_string, '%Y%m%d%H%M%S')

# Using the answers of question 2 and 3, we can determine up to what moment in the future the NWP has predictions
# the temporal resultion is hourly, and there are up 72 hours time steps into the future
# this means the Unified Model has a forecast horizon of 3 days ahead starting from the model time
# so we can determine what the last available datetime will be
last_model_timestamp = datasource_enddate + datetime.timedelta(days=3)


print(last_model_timestamp)


In [ ]:
# We know now that weather forecast data of the unified model is available up to 3 hours ahead
# the Unified Model runs twice per day, and provides a forecast for 3 days ahead

# it is also interesting to know in what area there is data available, as the Unified Model does not cover the entire globe
# the spatial extent of the UM is encapsulated in the grid definition in the TWIGA API
# you can find the grid definition in the JSON response above

# please fill in the griddefinition ID on the place of the questionmark
um_grid_definition_id = ?



In [ ]:
# we can now request additional information on the spatial extent from the API using the grid definition
# json request to ask metadata of a single datasource
request_grid_um = {
     "GridDefinitionIds": [um_grid_definition_id]
}

# Send the request to the griddefinitions endpoint of the API
griddef_um_response = requests.post(api + 'entity/griddefinitions/get', headers=api_header, data=json.dumps(request_grid_um))

# The response of the API is in JSON. Parse this with Python
griddef_um_metadata = griddef_um_response.json()

# print the result, as indented json
print(json.dumps(griddef_um_metadata, indent=2))

In [ ]:
# Optional: plot extent on a map

# in order to create some neat geogrpahic plots, python has the excellet cartopy package
# this package is not installed by default. Using an anacond prompt, try to run the following command
# conda install -c conda-forge cartopy

# when successfull, we can load the cartopy packages
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt

# get the XLL, XUR, YLL and YUR coordinates
um_xll = griddef_um_metadata["Extents"][str(um_grid_definition_id)].get("Xll")
um_xur = griddef_um_metadata["Extents"][str(um_grid_definition_id)].get("Xur")
um_yll = griddef_um_metadata["Extents"][str(um_grid_definition_id)].get("Yll")
um_yur = griddef_um_metadata["Extents"][str(um_grid_definition_id)].get("Yur")

# create a plot of the spatial extent of the Unified Model
plt.figure(figsize=(15, 9))
central_lon, central_lat = (um_xll + um_xur) / 2, (um_yll + um_yur) / 2
extent = [um_xll, um_xur, um_yll, um_yur]
ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
ax.set_extent(extent)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS)
ax.gridlines()

In [ ]:
# So far we have identified up to what point in time there is data available
# we know for what region/location we have data available
# the next step is to get look up for what variables there is data available


# again we define a JSON request which we can send the the TWIGA API
request_variables_um = {
     "DataSourceCodes": [selected_datasource_code]
}

# This time we send the request to the variables endpoint of the API
variables_um_response = requests.post(api + 'entity/variables/get', headers=api_header, data=json.dumps(request_variables_um))

# The response of the API is in JSON. Parse this with Python
variables_um_metadata = variables_um_response.json()

# print the result, as indented json
print(json.dumps(variables_um_metadata, indent=2))


# Questions

What variables are available? 
Is data from relativy humidity available?

In [ ]:
# Now we have all the information we need, and we can proceed to retrieve actual data from the TWIGA api

# first define a location, in lat/long. For example, Stellenbosch Jonkershoek Nature Reserve
lat = -33.979423
lon = 19.001196

# specify a variable of interest, for example temperature
# please note that you allways have to give the Code, this is the identifier used by the TWIGA platform
variable_code_of_interest = "TMP"

# We want to retrieve data of the latest available modelrun
# formatted as a string YYYYmmddHHMMSS
model_date = datasource_enddate.strftime('%Y%m%d%H%M%S')

# We want to retrieve all available time steps in the modelrun, so from the T=0 up to 3 days ahead
start_date = model_date
end_date = last_model_timestamp.strftime('%Y%m%d%H%M%S')


In [ ]:
# with all the information set, we can now build up the request we need to send to the TWIGA API
request_for_data_unified_model = {
     "TimeZoneOffset": "+0000",
     "Readers": [{
          "DataSourceCode": selected_datasource_code,
          "Settings": {
                "ModelDate": model_date,
                "StartDate": start_date,
                "EndDate": end_date,
                "VariableCodes": [variable_code_of_interest],
                "ReadAccumulated": "false",
                "Extent": {
                     "XLL": lon,
                     "YLL": lat,
                     "XUR": lon,
                     "YUR": lat,
                     "SpatialReference": {
                          "Epsg": "4326"
                     }
                }
          }
     }]
}


# send the request to the TWIGA API, this time the modelTimeseries endpoint
data_response = requests.post(api + 'modelTimeseries/get', headers=api_header, data=json.dumps(request_for_data_unified_model))

# parse response into JSON object
unified_model_data = data_response.json()

# print the result, as indented json
print(json.dumps(unified_model_data, indent=2))

In [ ]:
# the result is a nested JSON object, it contains both the meta data and the actual data of interest
# the actual data can be found in the 'Data' part of the JSON
# it can be ingested into a pandas dataframe

um_data = pd.DataFrame(unified_model_data['Data'][0]['Data'])

# convert the DateTime from string into datetime objects
um_data['DateTime'] = pd.to_datetime(um_data['DateTime'])

print(um_data)

In [ ]:
# rename the Value column to the name of the variable
um_data = um_data.rename(columns={"Value": variable_code_of_interest})

# we can use this data frame to create a plot of the data
um_data.plot(kind='line', x='DateTime', y=variable_code_of_interest, color='red')


In [ ]:
# We can also store this data to your local computer in a csv file
# please adjust the following line, to a valid location on your computer
csv_file_location = "C:/Some/Folder/temperature.csv"

# we can now save the data to a csv file on your local computer
um_data.to_csv("C:/Temp/temperature_data.csv",index=False)

In [ ]:
# previously we requested data of temperature, but there are more variables available
# we can use a loop to request data of multiple parameters
multiple_variables_of_interest = ["P", "TMP", "DPT", "TMIN", "TMAX", "WindSpeed", "WindDirection", "NCCC"]

# use a loop to retrieve data from the API
for i in range(len(multiple_variables_of_interest)):
    request_object = {
         "TimeZoneOffset": "+0000",
         "Readers": [{
              "DataSourceCode": selected_datasource_code,
              "Settings": {
                    "ModelDate": model_date,
                    "StartDate": start_date,
                    "EndDate": end_date,
                    "VariableCodes": [multiple_variables_of_interest[i]],
                    "ReadAccumulated": "false",
                    "Extent": {
                         "XLL": lon,
                         "YLL": lat,
                         "XUR": lon,
                         "YUR": lat,
                         "SpatialReference": {
                              "Epsg": "4326"
                         }
                    }
              }
         }]
    }
    # send the request to the TWIGA API
    data_response = requests.post(api + 'modelTimeseries/get', headers=api_header, data=json.dumps(request_object))
    unified_model_data = data_response.json()
    # extract the data into a dataframe
    um_values = pd.DataFrame(unified_model_data['Data'][0]['Data'])
    # rename the value column to the name of the variable
    um_values = um_values.rename(columns={"Value": multiple_variables_of_interest[i]})
    if i == 0:
        # convert the DateTime from string into datetime objects
        um_values['DateTime'] = pd.to_datetime(um_values['DateTime'])
        # create resulting dataframe in which to store all data
        um_data = um_values[["DateTime", multiple_variables_of_interest[i]]]
    else:
        # attach this column to the previously created data frame
        um_data = pd.concat([um_data, um_values[[multiple_variables_of_interest[i]]]], axis = 1)


In [ ]:
# In some cases there are -9999 values in the dataframe
# these values signal No Data values

# we can replace these -999 values with actual no data values in python
um_data.mask(um_data == -9999, inplace=True)

# print the resulting data frame
print(um_data)

# Questions
Can you figure out what the precipitation sum is?

What is the mean wind speed?

In [ ]:
# answer space




In [ ]:
# The Unified Model does not contain any data on relative humidity
# But the RH can be calculated using the temperature (T) and dewpoint temperature (TD)
def calc_relative_humidity(T, TD):
    RH = []
    b = 17.265
    c = 243.04
    for i in range(len(T)):
        RH_value = 100 * np.exp((c * b * (TD[i] - T[i])) / ( (c + T[i]) * (c + TD[i])))
        RH.append(RH_value)
    return RH
    

In [ ]:
# calculate the relative humidity
RH = calc_relative_humidity(T = um_data["TMP"].tolist(), TD = um_data["DPT"].tolist())

# attach the relative humidity to the data frame
um_data["RH"] = RH


In [ ]:
# print final data frame
print(um_data)

# Question
Can you get the weather forecast data for Nairobi?

Can you retrieve the solar radiation forecast for Johannesburg?


In [ ]:
# Answer space



# Grids
In all the examples above we have worked with time-series data of a singular gridcell/pixel
It is however also possible to retrieve grids from the TWIGA API

In [ ]:
# in order to request a grid, rather than a single pixel we can adjust a previous request

# previously when requestion data, we have the following request body
request_for_data_unified_model = {
     "TimeZoneOffset": "+0000",
     "Readers": [{
          "DataSourceCode": selected_datasource_code,
          "Settings": {
                "ModelDate": model_date,
                "StartDate": start_date,
                "EndDate": end_date,
                "VariableCodes": [variable_code_of_interest],
                "ReadAccumulated": "false",
                "Extent": {
                     "XLL": lon,
                     "YLL": lat,
                     "XUR": lon,
                     "YUR": lat,
                     "SpatialReference": {
                          "Epsg": "4326"
                     }
                }
          }
     }]
}

# In order to retrieve actual gridded data, we can simply omit the Extent from the request
# or adjust the values to get a part of the entire grid
# and we need to add that the structuretype is now a grid
request_griddata_unified_model = {
     "TimeZoneOffset": "+0000",
     "Readers": [{
          "DataSourceCode": selected_datasource_code,
          "Settings": {
                "StructureType": "ModelGrid",
                "ModelDate": model_date,
                "StartDate": start_date,
                "EndDate": end_date,
                "VariableCodes": [variable_code_of_interest],
                "ReadAccumulated": "false"
          }
     }]
}

# We also need to adjust the start or enddate
# Because we only want to request a single grid of one time step

last_model_timestamp = datasource_enddate + datetime.timedelta(days=3)
previous_timestamp = last_model_timestamp - datetime.timedelta(hours=1)

request_griddata_unified_model = {
     "TimeZoneOffset": "+0000",
     "Readers": [{
          "DataSourceCode": selected_datasource_code,
          "Settings": {
                "StructureType": "ModelGrid",
                "ModelDate": model_date,
                "StartDate": previous_timestamp.strftime('%Y%m%d%H%M%S'),
                "EndDate": last_model_timestamp.strftime('%Y%m%d%H%M%S'),
                "VariableCodes": [variable_code_of_interest],
                "ReadAccumulated": "false"
          }
     }]
}

# By default the TWIGA api will return data in json format
# but we can change this, to ask for a geotiff file instead
request_griddata_unified_model = {
     "TimeZoneOffset": "+0000",
     "Readers": [{
          "DataSourceCode": selected_datasource_code,
          "Settings": {
                "StructureType": "ModelGrid",
                "ModelDate": model_date,
                "StartDate": previous_timestamp.strftime('%Y%m%d%H%M%S'),
                "EndDate": last_model_timestamp.strftime('%Y%m%d%H%M%S'),
                "VariableCodes": [variable_code_of_interest],
                "ReadAccumulated": "false"
          }
     }],
     "Exporter": {
          "DataFormatCode": "geotiff"
     }
}




In [ ]:
# send the request to the TWIGA API, this time the data endpoint
data_grid_response = requests.post(api + 'data/get', headers=api_header, data=json.dumps(request_griddata_unified_model))

# the response now hold the geotiff file, save this to your local computer
store_geotiff = "C:/Temp/unified_model_tempetature.tif"
with open(store_geotiff, 'wb') as f:
    f.write(data_grid_response.content)



In [ ]:
# we can now load and plot the geotiff using the rasterio package
import rasterio

# load geotiff
src = rasterio.open(store_geotiff)
# plot the geotiff
plt.imshow(src.read(1), cmap='pink')
plt.show()

# Questions
Can you make a plot of the cloudcover, for tomorrow



In [ ]:
# answer space




# Questions
Can you make a plot of the precipitation sum of the entire 3 days?

Hint, add the following to the settings part of the request
			"Interval": {
				"Type": "Total",
				"Value": 0
			}

In [ ]:
# answer space




# Questions
Can you retrieve the precipitation sum of the Saws.Satellit product, for the month of october?
And can you also make a plot of the result?